In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pickle as pkl
import gc

In [3]:
sDir = '/home/pabhijit/data/'

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### Load Data

In [5]:
df_train = pd.read_csv(sDir + 'train.csv')
df_test = pd.read_csv(sDir + 'test.csv')
df_hist_trans = pd.read_csv(sDir + 'historical_transactions.csv',parse_dates=['purchase_date'])
df_new_merchant_trans = pd.read_csv(sDir + 'new_merchant_transactions.csv',parse_dates=['purchase_date'])

### Reduce Memory

In [6]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)
gc.collect()

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


0

In [7]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988
3,2017-09,C_ID_186d6a6901,4,3,0,0.142456
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790


In [8]:
df_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [9]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [10]:
df_new_merchant_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29


#### Max Purchase Date

In [11]:
max_date = df_new_merchant_trans['purchase_date'].max()
max_date

Timestamp('2018-04-30 23:59:59')

In [12]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [13]:
for df in [df_hist_trans,df_new_merchant_trans]:

    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['month_diff'] = ((max_date - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [14]:
cols = ['card_id','merchant_id','purchase_date','month_lag','year','weekofyear','month','day','dayofweek','weekend','month_diff']
df_hist_trans[cols].head()

,card_id,merchant_id,purchase_date,month_lag,year,weekofyear,month,day,dayofweek,weekend,month_diff
0,C_ID_4e6213e9bc,M_ID_e020e9b302,2017-06-25 15:33:07,-8,2017,25,6,25,6,1,2
1,C_ID_4e6213e9bc,M_ID_86ec983688,2017-07-15 12:10:45,-7,2017,28,7,15,5,1,2
2,C_ID_4e6213e9bc,M_ID_979ed661fc,2017-08-09 22:04:29,-6,2017,32,8,9,2,0,2
3,C_ID_4e6213e9bc,M_ID_e6d5ae8ea6,2017-09-02 10:06:26,-5,2017,35,9,2,5,1,3
4,C_ID_4e6213e9bc,M_ID_e020e9b302,2017-03-10 01:14:19,-11,2017,10,3,10,4,0,2


In [15]:
print('Columns - df_hist_trans :',df_hist_trans.columns)
print('Columns - df_new_merchant_trans :',df_new_merchant_trans.columns)

Columns - df_hist_trans : Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'year', 'weekofyear', 'month', 'day', 'dayofweek',
       'weekend', 'hour', 'month_diff'],
      dtype='object')
Columns - df_new_merchant_trans : Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'year', 'weekofyear', 'month', 'day', 'dayofweek',
       'weekend', 'hour', 'month_diff'],
      dtype='object')


In [16]:
def  get_agg_fea(count_df,prefix):
    aggs = {}
    for col in ['month','day','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    # aggs['month_diff'] = ['mean']
    aggs['month_diff'] = ['max','min','mean','var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']

    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',\
                'category_1','category_2','category_3','month_lag','card_id']:
        count_df[col+'_mean'] = count_df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean'] = ['mean']   

    new_columns = get_new_columns(prefix,aggs)
    count_df_gp = count_df.groupby('card_id').agg(aggs)
    count_df_gp.columns = new_columns
    count_df_gp.reset_index(drop=False,inplace=True)
    count_df_gp['%s_purchase_date_diff'%prefix] = (count_df_gp['%s_purchase_date_max'%prefix] - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    count_df_gp['%s_purchase_date_average'%prefix] = count_df_gp['%s_purchase_date_diff'%prefix]/count_df_gp['%s_card_id_size'%prefix]
    count_df_gp['%s_purchase_date_uptonow'%prefix] = (max_date - count_df_gp['%s_purchase_date_max'%prefix]).dt.days
    count_df_gp['%s_purchase_date_uptomin'%prefix] = (max_date - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    
    return count_df_gp

In [ ]:
hist_count_df=get_agg_fea(df_hist_trans,'hist')

In [ ]:
df_train = df_train.merge(hist_count_df,on='card_id',how='left')
df_test = df_test.merge(hist_count_df,on='card_id',how='left')
del hist_count_df
gc.collect()